# Inroduction about the dataset 

#The dataset is record of product sales and taxes associated with the products. Each row of the dataset represents a single transaction of a product sale, with the attributes providing information about the product being sold, the quantity sold, the rate at which it was sold, the subtotal, taxes applied, and the final total amount.

#Using the "originalrate" attribute as the target variable, we can develop machine learning models to predict the future rate of a product based on its sales history and other relevant factors. This can help businesses to optimize their pricing strategies and increase profitability, and  I want to use all the models which we tried in the class and find the best-fit model.


## Setup

In [20]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
# set random seed to ensure that results are repeatable
np.random.seed(1)

# Load the data

In [21]:
assign1=pd.read_csv(r"C:\Users\akhil\OneDrive - University of South Florida\Desktop\DSP\Assignment1\product.csv", encoding='ISO-8859-1')
assign1.head(3)

,productname,uom,qty,rate,amount,taxableamt,cgstrate,cgstamt,sgstrate,sgstamt,total,originalrate
0,671-LT,Nos,14,155.0,2170.0,2170.0,9.0,195.30,9.0,195.30,2560.6,155.0
1,673-HLA,Nos,1,155.0,155.0,155.0,9.0,13.95,9.0,13.95,182.9,155.0
2,GIALLO GRENE,Nos,3,230.0,690.0,690.0,9.0,62.10,9.0,62.10,814.2,230.0


### checking whether if there are null values

In [22]:
assign1.isna().sum()

productname     0
uom             0
qty             0
rate            0
amount          0
taxableamt      0
cgstrate        0
cgstamt         0
sgstrate        0
sgstamt         0
total           0
originalrate    0
dtype: int64

In [23]:
assign1.columns

Index(['productname', 'uom', 'qty', 'rate', 'amount', 'taxableamt', 'cgstrate',
       'cgstamt', 'sgstrate', 'sgstamt', 'total', 'originalrate'],
      dtype='object')

## One Hot Encoding

#One-hot encoding is typically used for categorical data, where each category is represented as a binary vector. 

#In the table provided, the only column that contains categorical data is the "uom" column, which represents the unit of measurement for each product.

#Therefore, we can do one-hot encoding for the "uom" column.

In [24]:
one_hot_encoding = pd.get_dummies(assign1[['uom']])

In [25]:
assign1 = assign1.join(one_hot_encoding)

In [26]:
#droping the uom column
assign1 = assign1.drop(['uom'],axis=1)

#Doing labelencoding for the productname Column for converting categorical data into numerical format by assigning a unique integer to each category.

In [27]:
labelencoder = LabelEncoder()
assign1['productname'] = labelencoder.fit_transform(assign1['productname'])

In [28]:
assign1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132 entries, 0 to 1131
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   productname   1132 non-null   int32  
 1   qty           1132 non-null   int64  
 2   rate          1132 non-null   float64
 3   amount        1132 non-null   float64
 4   taxableamt    1132 non-null   float64
 5   cgstrate      1132 non-null   float64
 6   cgstamt       1132 non-null   float64
 7   sgstrate      1132 non-null   float64
 8   sgstamt       1132 non-null   float64
 9   total         1132 non-null   float64
 10  originalrate  1132 non-null   float64
 11  uom_2         1132 non-null   uint8  
 12  uom_BAG       1132 non-null   uint8  
 13  uom_BOX       1132 non-null   uint8  
 14  uom_Box       1132 non-null   uint8  
 15  uom_NOS       1132 non-null   uint8  
 16  uom_Nos       1132 non-null   uint8  
dtypes: float64(9), int32(1), int64(1), uint8(6)
memory usage: 99.6 KB


In [29]:
assign1.head()

,productname,qty,rate,amount,taxableamt,cgstrate,cgstamt,sgstrate,sgstamt,total,originalrate,uom_2,uom_BAG,uom_BOX,uom_Box,uom_NOS,uom_Nos
0,27,14,155.0,2170.0,2170.0,9.0,195.30,9.0,195.30,2560.6,155.0,0,0,0,0,0,1
1,28,1,155.0,155.0,155.0,9.0,13.95,9.0,13.95,182.9,155.0,0,0,0,0,0,1
2,191,3,230.0,690.0,690.0,9.0,62.10,9.0,62.10,814.2,230.0,0,0,0,0,0,1
3,96,28,475.0,13300.0,13300.0,9.0,1197.00,9.0,1197.00,15694.0,475.0,0,0,0,0,0,1
4,201,2,255.0,510.0,510.0,9.0,45.90,9.0,45.90,601.8,255.0,0,0,0,0,0,1


In [30]:
assign1['originalrate'].describe()

count    1132.000000
mean      864.544643
std       962.922079
min        33.000000
25%       333.750000
50%       605.000000
75%       991.250000
max      7625.000000
Name: originalrate, dtype: float64

### Converting Continuous variable to categorical variable

#As we can see original rate is continuous target variable, to use the logistic regression, we are converting continuous "originalrate" variable into a categorical variable by binning it into multiple categories using a threshold value.

In [31]:
# Define the bin edges for the originalrate variable
bin_edges = [0, 333.75, 605, 991.25, 7625]
# Define the labels for each rate category
labels = [1, 2, 3, 4]

In [32]:
# Bin the originalrate variable into the specified intervals
assign1['rate_category'] = pd.cut(assign1['originalrate'], bins=bin_edges, labels=labels, include_lowest=True)

In [33]:
# Convert the rate_category variable to integer type
assign1['rate_category'] = assign1['rate_category'].astype(np.int64)

In [34]:
# Print the number of samples in each rate category
print(assign1['rate_category'].value_counts())

2    286
1    283
4    283
3    280
Name: rate_category, dtype: int64


In [35]:
#Now we are using rate_category as target variable and droping originalrate from the table

In [36]:
assign1 = assign1.drop('originalrate', axis=1)

In [38]:
assign1.head()

,productname,qty,rate,amount,taxableamt,cgstrate,cgstamt,sgstrate,sgstamt,total,uom_2,uom_BAG,uom_BOX,uom_Box,uom_NOS,uom_Nos,rate_category
0,27,14,155.0,2170.0,2170.0,9.0,195.30,9.0,195.30,2560.6,0,0,0,0,0,1,1
1,28,1,155.0,155.0,155.0,9.0,13.95,9.0,13.95,182.9,0,0,0,0,0,1,1
2,191,3,230.0,690.0,690.0,9.0,62.10,9.0,62.10,814.2,0,0,0,0,0,1,1
3,96,28,475.0,13300.0,13300.0,9.0,1197.00,9.0,1197.00,15694.0,0,0,0,0,0,1,2
4,201,2,255.0,510.0,510.0,9.0,45.90,9.0,45.90,601.8,0,0,0,0,0,1,1


# Spliting the data to train and test

In [39]:
train_df, test_df = train_test_split(assign1, test_size=0.3)


target = 'rate_category'
predictors = list(assign1.columns)
predictors.remove(target)

In [40]:
scaler = preprocessing.StandardScaler()
cols_to_stdize = ['qty', 'rate', 'amount', 'taxableamt', 'cgstrate',
       'cgstamt', 'sgstrate', 'sgstamt', 'total'] 
train_df[cols_to_stdize] = scaler.fit_transform(train_df[cols_to_stdize]) 


test_df[cols_to_stdize] = scaler.transform(test_df[cols_to_stdize]) 

In [41]:
train_X = train_df[predictors]
train_y = train_df[target] 
test_X = test_df[predictors]
test_y = test_df[target] 

train_df.to_csv('assign1_train_df.csv', index=False)
train_X.to_csv('assign1_train_X.csv', index=False)
train_y.to_csv('assign1_train_y.csv', index=False)
test_df.to_csv('assign1_test_df.csv', index=False)
test_X.to_csv('assign1_test_X.csv', index=False)
test_y.to_csv('assign1_test_y.csv', index=False)